In [3]:
import pandas as pd

df = pd.read_csv("CleanedWaitTime.csv")

df_filtered = df[df["Metric"].isin(["Volume", "50th Percentile"])]

pivot_df = df_filtered.pivot_table(
    index=["Province/territory", "Indicator", "Data year"],
    columns="Metric",
    values="Indicator result",
    aggfunc="first",
).reset_index()

pivot_df.columns.name = None
pivot_df = pivot_df.rename(
    columns={
        "50th Percentile": "Median Wait Time (Hours)",
        "Volume": "Procedure Volume",
    }
)

improvement_df = pivot_df[pivot_df["Data year"].isin([2013, 2023])]

year_pivot = improvement_df.pivot_table(
    index=["Province/territory", "Indicator"],
    columns="Data year",
    values="Median Wait Time (Hours)",
    aggfunc="first",
).reset_index()

year_pivot.columns.name = None
year_pivot = year_pivot.rename(columns={2013: "Wait Time 2013", 2023: "Wait Time 2023"})
year_pivot["Wait Time Improvement"] = (
    year_pivot["Wait Time 2013"] - year_pivot["Wait Time 2023"]
)

final_df = pd.merge(
    pivot_df,
    year_pivot[["Province/territory", "Indicator", "Wait Time Improvement"]],
    on=["Province/territory", "Indicator"],
    how="left",
)

final_df.to_csv("PivotedWaitTimeData.csv", index=False)
